# Simulation of the Process with Input Variations

Import PyRolL modules.

In [1]:
import pyroll.basic as pr
import pyroll.export as pre

Import the process data.

In [2]:
from weiner_variation.sim.process import PASS_SEQUENCE, IN_PROFILE, TEMPERATURE, DIAMETER, TEMPERATURE_STD, DIAMETER_STD, create_in_profile
from weiner_variation.sim.config import SAMPLE_COUNT, SEED
from weiner_variation.sim.data import DrawInput
from weiner_variation.config import DATA_DIR

Import further libraries.

In [3]:
import pandas as pd
from copy import deepcopy
from scipy.stats import norm
import numpy as np
import tqdm
from multiprocessing import Pool

Parameters for notebook using papermill.

In [4]:
DIAMETER_STD = DIAMETER_STD
TEMPERATURE_STD = TEMPERATURE_STD
OUTPUT_FILENAME = DATA_DIR / "sim_input_results.csv"

In [5]:
# Parameters
OUTPUT_FILENAME = "/home/maxwe/Git/pyroll/publication-weiner-variation/weiner_variation/data/sim_diameter_stds_results/0.05.csv"
DIAMETER_STD = 0.0025000000000000005


Create distribution functions for input values.

In [6]:
diameter_dist = norm(loc=DIAMETER, scale=DIAMETER_STD)
temperature_dist = norm(loc=TEMPERATURE, scale=TEMPERATURE_STD)

Draw random inputs from distributions.

In [7]:
RNG = np.random.default_rng(SEED)
diameters = diameter_dist.rvs(random_state=RNG, size=SAMPLE_COUNT)
temperatures = temperature_dist.rvs(random_state=RNG, size=SAMPLE_COUNT)

draws = [
    DrawInput(d, t)
    for d, t in zip(diameters, temperatures)
]

Define a worker function creating the in profile, running the solution procedure and extracting results.

In [8]:
def worker(draw: DrawInput):    
    ip = create_in_profile(draw.diameter)
    ip.temperature = draw.temperature

    sequence = deepcopy(PASS_SEQUENCE)
    sequence.solve(ip)       

    return pre.to_pandas(sequence)

Run the simulations using a process pool.

In [9]:
results = list(tqdm.tqdm(Pool().imap(worker, draws), total=SAMPLE_COUNT))

  0%|          | 0/10 [00:00<?, ?it/s]

 10%|█         | 1/10 [00:00<00:06,  1.46it/s]

 60%|██████    | 6/10 [00:00<00:00,  9.35it/s]

 90%|█████████ | 9/10 [00:01<00:00,  7.85it/s]

100%|██████████| 10/10 [00:01<00:00,  7.81it/s]

Create a dataframe from results.

In [10]:
df = pd.DataFrame([r.stack().swaplevel().sort_index() for r in results])
df

abs_draught                                                              \
           0         2         4         6         8         10        12   
0   -0.015779 -0.020702 -0.014453 -0.019883 -0.009268 -0.011218 -0.007985   
1   -0.016482 -0.021404 -0.014493 -0.019903 -0.009272 -0.011220 -0.007985   
2   -0.017488 -0.022391 -0.014544 -0.019928 -0.009277 -0.011222 -0.007985   
3   -0.015046 -0.019947 -0.014405 -0.019859 -0.009264 -0.011216 -0.007984   
4   -0.016509 -0.021431 -0.014495 -0.019904 -0.009272 -0.011220 -0.007985   
5   -0.017311 -0.022218 -0.014536 -0.019924 -0.009276 -0.011222 -0.007985   
6   -0.015426 -0.020342 -0.014431 -0.019871 -0.009266 -0.011217 -0.007984   
7   -0.015336 -0.020249 -0.014425 -0.019868 -0.009266 -0.011217 -0.007984   
8   -0.011696 -0.015739 -0.014004 -0.019654 -0.009224 -0.011198 -0.007981   
9   -0.014676 -0.019546 -0.014378 -0.019845 -0.009261 -0.011215 -0.007984   

                                 ... zouhar_contact_in_width            \
         14        16        18  ...                      8         10   
0 -0.010435 -0.004337 -0.005911  ...                   0.026  0.005701   
1 -0.010435 -0.004337 -0.005911  ...                   0.026  0.005700   
2 -0.010435 -0.004337 -0.005911  ...                   0.026  0.005699   
3 -0.010435 -0.004337 -0.005911  ...                   0.026  0.005702   
4 -0.010435 -0.004337 -0.005911  ...                   0.026  0.005700   
5 -0.010435 -0.004337 -0.005911  ...                   0.026  0.005699   
6 -0.010435 -0.004337 -0.005911  ...                   0.026  0.005701   
7 -0.010435 -0.004337 -0.005911  ...                   0.026  0.005702   
8 -0.010433 -0.004336 -0.005911  ...                   0.026  0.005710   
9 -0.010435 -0.004337 -0.005911  ...                   0.026  0.005703   

                                                                        
       12        14      16        18     20        22    24        26  
0  0.0198  0.003251  0.0148  0.004102  0.012  0.003359  0.01  0.002245  
1  0.0198  0.003251  0.0148  0.004102  0.012  0.003359  0.01  0.002245  
2  0.0198  0.003251  0.0148  0.004102  0.012  0.003359  0.01  0.002245  
3  0.0198  0.003252  0.0148  0.004102  0.012  0.003359  0.01  0.002245  
4  0.0198  0.003251  0.0148  0.004102  0.012  0.003359  0.01  0.002245  
5  0.0198  0.003251  0.0148  0.004102  0.012  0.003359  0.01  0.002245  
6  0.0198  0.003251  0.0148  0.004102  0.012  0.003359  0.01  0.002245  
7  0.0198  0.003251  0.0148  0.004102  0.012  0.003359  0.01  0.002245  
8  0.0198  0.003253  0.0148  0.004102  0.012  0.003359  0.01  0.002245  
9  0.0198  0.003252  0.0148  0.004102  0.012  0.003359  0.01  0.002245  

[10 rows x 3891 columns]

Save dataframe to CSV file.

In [11]:
df.to_csv(OUTPUT_FILENAME)